# 05-xrefs
Get xrefs from a variety of sources
- Drugs: 
UMLS has mesh xrefs. From mesh, we can get UNII and CAS. From UNII_FDA, we can get inchikeys 
(lookup using cas or unii). From chembl, we can get chembl IDs from the inchikeys
So: UMLS -> mesh -> unii/cas -> inchikey -> chembl
insane, I know.
- Anatomy: uberon has umls xrefs
- disease: DO has umls, umls has NCI, ICD10PCS, SNOMEDCT_US, ICD10CM, OMIM
- proteins: umls has uniprot xrefs
- biological_process_or_activity/activity_and_behavior: umls has GO
- gene: umls has HGNC and OMIM

In [1]:
import sys
import os
import pickle
%matplotlib inline
import pandas as pd
from numpy import nan
import seaborn as sns
import shelve
import re
from collections import defaultdict, Counter
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from itertools import chain
from more_itertools import chunked
from collections import Counter
from pprint import pprint
import requests
from pyquery import PyQuery as pq
from wikidataintegrator import wdi_helpers, wdi_core, wdi_login
from semmed_biolink_environment import *

In [2]:
uri_to_curie = lambda s: s.split("/")[-1].replace("_", ":")

In [3]:
nodes = pd.read_csv(NODES_BIOLINK_TSV, sep='\t', index_col=0)

In [4]:
nodes.head()

,LABEL,umls_type,umls_type_label,blm_category
ID,,,,
C1516083,Attachment Plaque,T026,Cell Component,CellularComponent
C0656503,"4-nonyl-3,5-diethoxycarbonyl-1,4-dihydro-2,6-d...",T121|T109,Organic Chemical|Pharmacologic Substance,ChemicalSubstance
C0234143,Neurological muscle weakness,T047,Disease or Syndrome,DiseaseOrPhenotypicFeature
C0861091,Methanol increased,T033,Finding,DiseaseOrPhenotypicFeature
C0074127,scarlet red,T121|T109|T130,Organic Chemical|Pharmacologic Substance|Indic...,ChemicalSubstance


In [5]:
nodes.blm_category.value_counts()

ChemicalSubstance              58700
DiseaseOrPhenotypicFeature     36958
Protein                        28019
Gene                           23788
GrossAnatomicalStructure        9086
BiologicalProcessOrActivity     7665
AnatomicalEntity                2757
CellularComponent               1732
Cell                            1255
ActivityAndBehavior              881
PhenotypicFeature                418
GenomicEntity                    172
Name: blm_category, dtype: int64

## parse UMLS flat file to get all UMLS xrefs
# see: https://www.ncbi.nlm.nih.gov/books/NBK9685/

In [6]:
names = "CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X".split(",")
umls = pd.read_csv(MRCONSO_ENG_ARCHIVE, delimiter="|", names=names, index_col=None)
# only get CUIs in our list of nodes
umls = umls[umls.CUI.isin(nodes.index)]

/Users/richard/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
umls['xref'] = umls.SAB + ":" + umls.CODE.map(str)
# easy fix to HGNC prefix duplication between SAB and CODE
umls.xref = umls.xref.str.replace("HGNC:HGNC:", "HGNC:")
# fix this MSH MESH nonsense
umls.xref = umls.xref.str.replace("MSH:", "MESH:")
# NCI_FDA is UNII
umls.xref = umls.xref.str.replace("NCI_FDA:", "UNII:")

In [8]:
umls.head(10)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X,xref
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,RXNORM:1926948
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,MTH:NOCODE
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,NaN,M0023172,D015060,MSH,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",0,N,NaN,NaN,MESH:D015060
5,C0000039,ENG,P,L0000039,VC,S1357296,Y,A1317708,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dipalmitoylphosphatidylcholine",0,N,NaN,NaN,MESH:D015060
6,C0000039,ENG,S,L0000035,PF,S0007560,Y,A26674543,NaN,M0023172,D015060,MSH,ET,D015060,"1,2-Dihexadecyl-sn-Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
7,C0000039,ENG,S,L0000035,VO,S1357276,Y,A1317687,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dihexadecyl sn Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
8,C0000039,ENG,S,L0000038,PF,S0007563,Y,A26661070,NaN,M0023172,D015060,MSH,ET,D015060,"1,2-Dipalmitoyl-Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
9,C0000039,ENG,S,L0000038,VO,S1357295,Y,A1317707,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dipalmitoyl Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
10,C0000039,ENG,S,L0012507,PF,S0033298,N,A18399186,NaN,LP15542-1,NaN,LNC,LPN,LP15542-1,Dipalmitoylphosphatidylcholine,0,N,256.0,NaN,LNC:LP15542-1
11,C0000039,ENG,S,L0012507,PF,S0033298,N,A22817493,166113012.0,102735002,NaN,SNOMEDCT_US,OAP,102735002,Dipalmitoylphosphatidylcholine,9,O,256.0,NaN,SNOMEDCT_US:102735002


In [9]:
XREF = dict(umls.groupby("CUI")['xref'].apply(set))
XREF = defaultdict(set, XREF)
print(XREF['C0000039'])

{'SNOMEDCT_US:102735002', 'RXNORM:1926948', 'LNC:LP15542-1', 'MESH:D015060', 'MTH:NOCODE', 'LNC:MTHU010538'}


### Chemicals and drugs

In [10]:
# what xrefs are on chemicals?
chem_umls = nodes[nodes.blm_category == "ChemicalSubstance"].index
xref_chem = {k:v for k,v in XREF.items() if k in chem_umls}
print(len(chem_umls))
c = Counter(list(chain(*[list(map(lambda x:x.split(":",1)[0], y)) for y in xref_chem.values()])))
pprint(c.most_common(25))
# nearly all have a mesh ID. not much of anything else
# neither mesh nor umls have inchikeys, or inchi, or smiles or anything useful for linking out
# blech

58700
[('MESH', 51182),
 ('SNOMEDCT_US', 15435),
 ('NCI', 11047),
 ('RXNORM', 9143),
 ('CHV', 8331),
 ('MMSL', 8046),
 ('MTH', 7294),
 ('MTHSPL', 6649),
 ('UNII', 6573),
 ('NDDF', 6371),
 ('LNC', 5558),
 ('VANDF', 4040),
 ('DRUGBANK', 3496),
 ('GS', 3406),
 ('ATC', 3204),
 ('PDQ', 2965),
 ('MEDCIN', 2914),
 ('CSP', 2896),
 ('USP', 1887),
 ('LCH_NW', 1826),
 ('NCI_NCI-GLOSS', 1505),
 ('NCI_CTRP', 1007),
 ('USPMG', 939),
 ('NCI_DTP', 834),
 ('NCI_DCP', 734)]


In [11]:
pd.set_option("display.width", 120)

URL = "http://id.nlm.nih.gov/mesh/sparql"
PREFIX = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
"""

def sparql_query(query):
    params = {'query': PREFIX + query, 'format': 'JSON', 'limit': 1000, 'offset': 0}
    r = requests.get(URL, params=params)
    res = [{k: v['value'] for k, v in x.items()} for x in r.json()['results']['bindings']]
    t = tqdm()
    while True:
        t.update(1)
        params['offset'] += 1000
        r = requests.get(URL, params=params).json()['results']['bindings']
        if not r:
            break
        res.extend([{k: v['value'] for k, v in x.items()} for x in r])
    df = pd.DataFrame(res)
    return df

In [12]:
query = """
SELECT distinct ?mesh ?meshLabel ?r ?rr
FROM <http://id.nlm.nih.gov/mesh> WHERE {
  ?mesh meshv:active 1 .
  ?mesh meshv:preferredMappedTo ?p .
  ?p meshv:treeNumber ?treeNum .
  FILTER(STRSTARTS(STR(?treeNum), "http://id.nlm.nih.gov/mesh/D")) .
  ?mesh rdfs:label ?meshLabel .
  ?mesh meshv:preferredConcept [meshv:registryNumber ?r] .
  #OPTIONAL {?mesh meshv:preferredConcept [meshv:relatedRegistryNumber ?rr]}
}
"""
df = sparql_query(query)

218it [05:41,  1.74s/it]

In [13]:
df.r = df.r.replace("0", nan)
df.dropna(subset=["r"], inplace=True)
df = df[~df.r.str.startswith("EC ")]
df.mesh = df.mesh.str.replace("http://id.nlm.nih.gov/mesh/", "")
df.set_index("mesh", inplace=True)

In [14]:
df.to_csv(MESH_XREFS_TSV, sep='\t')
df.head()

,meshLabel,r
mesh,,
C000629587,boron tribromide,A453DV9339
C016271,bromine chloride,7G62XY5724
C016769,Syn-ergel,66799-40-4
C017138,CMB-dextran,37307-31-6
C017827,bacterio-opsin,54577-62-7


In [15]:
mesh_xrefs = pd.read_csv(MESH_XREFS_TSV, sep='\t', index_col=0)
mesh_xrefs.r = mesh_xrefs.r.apply(lambda x: "CAS:" + x if "-" in x else "UNII:" + x)
mesh_xrefs = mesh_xrefs.groupby("mesh").r.apply(set).to_dict()
mesh_xrefs = {"MESH:"+k:v for k,v in mesh_xrefs.items()}
len(mesh_xrefs)

40189

In [16]:
for k,v in xref_chem.items():
    for vv in list(v):
        if vv in mesh_xrefs:
            v.update(mesh_xrefs[vv])

In [17]:
# download: 'http://fdasis.nlm.nih.gov/srs/download/srs/UNII_Data.zip'
unii_df = pd.read_csv(UNII_RECORDS, dtype=str, sep='\t', low_memory=False)
unii_df.dropna(subset=['INCHIKEY'], inplace=True)

In [18]:
unii_df.head()

,UNII,PT,RN,EC,NCIT,RXCUI,PUBCHEM,ITIS,NCBI,PLANTS,GRIN,MPNS,INN_ID,MF,INCHIKEY,SMILES,INGREDIENT_TYPE
0,00174624E2,CHF-6333 CATION,1613620-10-2,NaN,NaN,NaN,76285164,NaN,NaN,NaN,NaN,NaN,NaN,C27H28F3N6O3,IHTRPSMRGYWUIM-HSZRJFAPSA-O,COC(=O)C1=C(C)N(C2=NNC(=O)N2[C@@H]1C3=CC=C(C=C...,IONIC MOIETY
1,0129526470,"5,8-DIMETHOXY(1,2,4)TRIAZOLO(1,5-C)PYRIMIDIN-2...",219715-62-5,NaN,NaN,NaN,11446888,NaN,NaN,NaN,NaN,NaN,NaN,C7H9N5O2,DBJPBHJHAPAUQU-UHFFFAOYSA-N,COC1=CN=C(OC)N2N=C(N)N=C12,INGREDIENT SUBSTANCE
3,0377415922,"N-DESMETHYLVENLAFAXINE, (S)-",392332-59-1,NaN,NaN,NaN,9860056,NaN,NaN,NaN,NaN,NaN,NaN,C16H25NO2,MKAFOJAJJMUXLW-OAHLLOKOSA-N,CNC[C@H](C1=CC=C(OC)C=C1)C2(O)CCCCC2,INGREDIENT SUBSTANCE
5,0480546720,HOMOCYCLOLEUCINE HYDROCHLORIDE,39692-17-6,254-594-3,NaN,NaN,2724466,NaN,NaN,NaN,NaN,NaN,NaN,C7H13NO2.ClH,GTKXSYHXQSKWNP-UHFFFAOYSA-N,Cl.NC1(CCCCC1)C(O)=O,INGREDIENT SUBSTANCE
6,0503177591,SUCCINALDEHYDE,638-37-9,211-333-8,NaN,NaN,12524,NaN,NaN,NaN,NaN,NaN,NaN,C4H6O2,PCSMJKASWLYICJ-UHFFFAOYSA-N,O=CCCC=O,INGREDIENT SUBSTANCE


In [19]:
n=0
for k,v in tqdm_notebook(xref_chem.items()):
    for vv in list(v):
        if vv.startswith("UNII:"):
            xref = vv.replace("UNII:", "")
            s = unii_df.query("UNII == @xref").INCHIKEY
            if not s.empty:
                n+=1
                v.add("INCHIKEY:" + list(s)[0])

In [20]:
xref_inchi = {k:v for k,v in xref_chem.items() if any(vv.startswith("INCHIKEY:") for vv in v)}
xref_inchi = {k:[vv for vv in v if vv.startswith("INCHIKEY:")][0].replace("INCHIKEY:", "") for k,v in xref_inchi.items()}
print(len(xref_inchi))
list(xref_inchi.items())[:4]

10188


[('C0000248', 'CZIHNRWJTSTCEX-UHFFFAOYSA-N'),
 ('C0000294', 'XOGTZOOQQBDUSI-UHFFFAOYSA-M'),
 ('C0000378', 'QXWYKJLNLSIPIN-JGVFFNPUSA-N'),
 ('C0000399', 'CBOKZNLSFMZJJA-PEBGCTIMSA-N')]

In [21]:
url = "https://www.ebi.ac.uk/chembl/api/data/molecule?molecule_structures__standard_inchi_key__in={}&format=json&limit=100"
for chunk in tqdm(chunked(xref_inchi.items(), 100), total=len(xref_inchi)/100):
    chunk = dict(chunk)
    chunk = {v:k for k,v in chunk.items()}
    inchis = ",".join(chunk)
    mols = requests.get(url.format(inchis)).json()['molecules']
    for m in mols:
        chembl = m['molecule_chembl_id']
        inchi = m['molecule_structures']['standard_inchi_key']
        XREF[chunk[inchi]].add("CHEMBL:" + chembl)


  0%|          | 0/101.88 [00:00<?, ?it/s]
  1%|          | 1/101.88 [00:03<05:37,  3.34s/it]
  2%|▏         | 2/101.88 [00:07<05:54,  3.55s/it]
  3%|▎         | 3/101.88 [00:10<05:52,  3.56s/it]
  4%|▍         | 4/101.88 [00:19<08:13,  5.04s/it]
  5%|▍         | 5/101.88 [00:26<09:04,  5.62s/it]
  6%|▌         | 6/101.88 [00:32<09:23,  5.88s/it]
  7%|▋         | 7/101.88 [00:41<10:24,  6.58s/it]
  8%|▊         | 8/101.88 [00:48<10:48,  6.91s/it]
  9%|▉         | 9/101.88 [00:54<10:20,  6.69s/it]
 10%|▉         | 10/101.88 [01:02<10:41,  6.98s/it]
 11%|█         | 11/101.88 [01:10<11:03,  7.30s/it]
 12%|█▏        | 12/101.88 [01:22<12:49,  8.56s/it]
 13%|█▎        | 13/101.88 [01:26<10:58,  7.41s/it]
 14%|█▎        | 14/101.88 [01:31<09:44,  6.65s/it]
 15%|█▍        | 15/101.88 [01:35<08:30,  5.88s/it]
 16%|█▌        | 16/101.88 [01:41<08:23,  5.86s/it]
 17%|█▋        | 17/101.88 [01:45<07:32,  5.33s/it]
 18%|█▊        | 18/101.88 [01:51<07:48,  5.58s/it]
 19%|█▊        | 19/101.88 [0

In [22]:
len({k:v for k,v in XREF.items() if any(vv.startswith("CHEMBL:") for vv in v)})

8105

In [23]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

## UBERON, uses uberon.csv generated from the uberon.owl file previously downloaded (see README)

In [24]:
df = pd.read_csv(UBERON_CSV)
df = df[df.xref.str.startswith("UMLS:")]
df.xref = df.xref.str.replace("UMLS:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
9755,UBERON:0007227,C0228757
11788,UBERON:0002965,C0175263
11853,UBERON:0001532,C0007276
11897,UBERON:0014636,C0228627
11904,UBERON:0002610,C0152411


In [25]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [26]:
XREF['C1272528']

{'MTH:NOCODE', 'SNOMEDCT_US:384765009', 'UBERON:0006472'}

## DOID, uses doid.csv generated from the doi.owl file previously downloaded (see README)

In [27]:
df = pd.read_csv(DOID_CSV)
df.dropna(inplace=True)
df = df[df.xref.str.startswith("UMLS_CUI:")]
df.xref = df.xref.str.replace("UMLS_CUI:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
6,DOID:3864,C1332188
7,DOID:3864,C0278876
21,DOID:699,C0162670
27,DOID:0111831,C1844936
34,DOID:0111618,C1853116


In [28]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [29]:
XREF['C0263518']

{'DOID:1943',
 'HPO:HP:0025470',
 'ICD10:L65.0',
 'ICD10AM:L65.0',
 'ICD10CM:L65.0',
 'ICD9CM:704.02',
 'ICPC2ICD10ENG:MTHU025078',
 'ICPC2ICD10ENG:MTHU073653',
 'MDR:10043200',
 'MEDCIN:37106',
 'MTH:NOCODE',
 'NCI:C112200',
 'NCI_NICHD:C112200',
 'SNOMEDCT_US:201147004',
 'SNOMEDCT_US:39479004'}

In [30]:
XREF['C0591520']

{'CHV:0000041179', 'MESH:D000068298'}

## proteins

In [31]:
# See README regarding MRSAT_ARCHIVE
names = list("abcdefghijklmn")
iter_csv = pd.read_csv(MRSAT_ARCHIVE, delimiter="|", names=names, index_col=None, chunksize=1000000)
chunks = []
umls_uniprot = dict()
for chunk in tqdm(iter_csv, total=67668372/1000000):
    chunk.fillna(method='ffill', inplace=True)
    chunk = chunk[chunk.i == "SWISS_PROT"]
    d = dict(zip(chunk.a, chunk.k))
    umls_uniprot.update(d)


  0%|          | 0/67.668372 [00:00<?, ?it/s]
  1%|▏         | 1/67.668372 [00:03<03:44,  3.36s/it]/Users/richard/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2,5,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):

  3%|▎         | 2/67.668372 [00:06<03:33,  3.25s/it]
  4%|▍         | 3/67.668372 [00:09<03:24,  3.16s/it]/Users/richard/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):

  6%|▌         | 4/67.668372 [00:12<03:18,  3.11s/it]
  7%|▋         | 5/67.668372 [00:15<03:16,  3.14s/it]
  9%|▉         | 6/67.668372 [00:18<03:13,  3.14s/it]
 10%|█         | 7/67.668372 [00:21<03:11,  3.16s/it]
 12%|█▏        | 8/67.668372 [00:24<03:08,  3.15s/it]
 13%|█

In [32]:
len(umls_uniprot)

4916

In [33]:
for umls, uniprot in umls_uniprot.items():
    XREF[umls].add("UNIPROT:" + uniprot)

In [34]:
XREF['C0215993']

{'MESH:C081092',
 'MTH:NOCODE',
 'NCI:C127008',
 'NCI_CTRP:C127008',
 'UNIPROT:Q04756'}

In [35]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

In [36]:
nodes['xrefs'] = nodes.index.map(lambda x: ";".join(XREF.get(x,list())))

In [37]:
nodes.head(20)

,LABEL,umls_type,umls_type_label,blm_category,xrefs
ID,,,,,
C1516083,Attachment Plaque,T026,Cell Component,CellularComponent,NCI:C32163
C0656503,"4-nonyl-3,5-diethoxycarbonyl-1,4-dihydro-2,6-d...",T121|T109,Organic Chemical|Pharmacologic Substance,ChemicalSubstance,MESH:C078953
C0234143,Neurological muscle weakness,T047,Disease or Syndrome,DiseaseOrPhenotypicFeature,SNOMEDCT_US:41786007;SNOMEDCT_VET:41786007
C0861091,Methanol increased,T033,Finding,DiseaseOrPhenotypicFeature,MDR:10027503;MDR:10027504
C0074127,scarlet red,T121|T109|T130,Organic Chemical|Pharmacologic Substance|Indic...,ChemicalSubstance,UNII:I35E9QU96C;SNOMEDCT_US:412509006;CHV:0000...
C0048423,4-isopropylbicyclophosphate,T121|T109,Organic Chemical|Pharmacologic Substance,ChemicalSubstance,MESH:C008207;CAS:51052-72-3
C0878406,CT-1341,T121|T109,Organic Chemical|Pharmacologic Substance,ChemicalSubstance,MESH:D000530
C1540039,TOR1A gene,T028,Gene or Genome,Gene,LNC:LP31868-0;OMIM:605204;HGNC:3098;MTH:NOCODE...
C0058391,diphenylthiourea,T130|T109,"Organic Chemical|Indicator, Reagent, or Diagno...",ChemicalSubstance,SNOMEDCT_US:256150005;SNOMEDCT_US:256116001;SN...


In [38]:
nodes.to_csv(NODES_XREF_TSV, sep='\t')